In [ ]:
#pip install -U transformer

In [ ]:
#pip install 'accelerate>=0.26.0'

In [ ]:
# simple_bert_finetune_compat.py
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate

# 1) dataset (replace with your CSV if you want)
dataset = load_dataset("glue", "sst2")  # toy binary sentiment

# 2) tokenizer and tokenize
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_fn(batch):
    return tokenizer(batch["sentence"], truncation=True, padding="max_length", max_length=128)

dataset = dataset.map(tokenize_fn, batched=True)

# 3) model (classifier head will be randomly initialized — that's expected)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 4) metric
acc = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return acc.compute(predictions=preds, references=labels)

# 5) training args — compatible with older transformers (no evaluation_strategy)
training_args = TrainingArguments(
    output_dir="./bert-simple",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_steps=200,
    save_steps=500
)

# 6) trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# 7) train
trainer.train()

# 8) save
trainer.save_model("./bert-simple")
tokenizer.save_pretrained("./bert-simple")
